In [1]:
# You may need to remove the '#' from the following commands to install the required dependencies

#these two are to read the excel
#! pip install xlrd
#! pip install install openpyxl

#These are to run R

In [2]:
# Importing libraries
import pandas as pd
import os
import numpy as np

import statsmodels.formula.api as smf
from itertools import combinations

import time

In [3]:
min_repetitions=2

In [4]:
#One row per cell line
x = pd.read_excel('data/ProCan-DepMapSanger_protein_matrix_6692_averaged.xlsx', engine='openpyxl').fillna(0).drop(columns=['Project_Identifier'])
c = [a.replace(';','.') for a in x.columns]
x.columns = c
x.columns

Index(['Cell_Line', 'P37108.SRP14_HUMAN', 'Q96JP5.ZFP91_HUMAN',
       'Q9Y4H2.IRS2_HUMAN', 'P36578.RL4_HUMAN', 'Q6SPF0.SAMD1_HUMAN',
       'O76031.CLPX_HUMAN', 'Q8WUQ7.CATIN_HUMAN', 'A6NIH7.U119B_HUMAN',
       'Q9BTD8.RBM42_HUMAN',
       ...
       'P33151.CADH5_HUMAN', 'Q5EBL4.RIPL1_HUMAN', 'P49715.CEBPA_HUMAN',
       'Q5TA45.INT11_HUMAN', 'O14924.RGS12_HUMAN', 'Q7Z3B1.NEGR1_HUMAN',
       'O60669.MOT2_HUMAN', 'Q13571.LAPM5_HUMAN', 'Q96JM2.ZN462_HUMAN',
       'P35558.PCKGC_HUMAN'],
      dtype='object', length=6693)

In [5]:
y = pd.read_csv('data/DrugResponse_PANCANCER_GDSC1_GDSC2_20200602.csv')[['drug_id','cell_line_name','ln_IC50']]
y.columns

/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['drug_id', 'cell_line_name', 'ln_IC50'], dtype='object')

In [6]:
# This cell is the function to go from the table to a JSON file (variantSpark format and structure)

def merge_tree_node(tree, node):
    
    # Empty tree
    if type(tree)==float: return tree
    if len(tree)==0: return node

    # Direct children
    if tree['right'] == node['nodeID']:
        tree['right'] = node
        return tree
    elif tree['left'] == node['nodeID']:
        tree['left'] = node
        return tree

    # Create
    right = merge_tree_node(tree['right'], node)
    left = merge_tree_node(tree['left'], node)
    tree['right'] = right
    tree['left'] = left
    return tree
            

def from_table_to_json(m):
    tree = {}
    for _id,row in m.iterrows():
        current_node = {'nodeID': row['nodeID'], 
                        'splitvarID':row['splitvarID'],
                        'splitVar':row['splitvarName'],
                        'splitval':row['splitval'], 
                        'terminal':row['terminal'], 
                        'prediction':row['prediction'], 
                        'left':row['leftChild'], 
                        'right':row['rightChild'] }
        tree = merge_tree_node(tree, current_node)
    return tree



# Test
#m = pd.read_csv('output/tree1.csv')
#from_table_to_json(m)

In [7]:
# Algorithm to get the interacting nodes (no testing done yet)

def get_interactions(tree, current_list, interactions):
    if not 'splitVar' in tree.keys():
        return 0
    if str(tree['splitVar']) == 'nan': return 0 #ranger adds a fake predicting node at the end
    
    # Adding the interaction
    current_list.append(tree['splitVar'])
    if len(current_list) >= 2:
        for i in range(2,len(current_list)+1):
            aux = '+'.join(sorted(current_list[-i:]))
            if aux in interactions.keys():
                interactions[aux] +=1
            else:
                interactions[aux] = 1
                    
    if 'left' in tree.keys():
        get_interactions(tree['left'], current_list, interactions)
    if 'right' in tree.keys():
        get_interactions(tree['right'], current_list, interactions)
        
    _ = current_list.pop()
    

In [8]:
# Testing all the interactions

def test_interactions(df, data):
    """
    I use GLM because:
    The main difference between the two approaches is that the general linear model strictly assumes that
    the residuals will follow a conditionally normal distribution, while the GLM loosens this assumption 
    and allows for a variety of other distributions from the exponential family for the residuals.
    """
    final_results = []
    counts = 0

    for v in df[(df.repetitions>=2) & (df.order ==2)].variants.tolist():
        
        #preparing the input
        sp=v.split('+')
        xy = data[sp+['ln_IC50']].fillna(-1)
        sp=v.replace('_','').split('+')
        xy.columns = [''.join([chr(int(y)+97) if y.isnumeric() else y for y in x.replace('_','').replace('.','')]) for x in xy.columns]
        formula = xy.columns[-1]+' ~ '
        for i in range(1,len(xy.columns)):
            formula = formula + ' + '.join(['*'.join(o) for o in list(combinations(xy.columns[:-1],i))])
            formula = formula + ' + '
        formula = formula.rstrip(' + ')

        # Standard fitting
        ols = smf.ols(formula,data=xy)
        ols.raise_on_perfect_prediction = False #preventing the perfect separation error
        results = ols.fit(disp=False, maxiter=1000) #mehtod prevents singular matrix
        results = results.summary()
        converged = results.tables[0].data[5][1].strip()
        pseudo_r2 = results.tables[0].data[3][3].strip()
        results = results.tables[1].data
        results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
        results['standard_fitting'] = True

        #If nan means no convergence bc singular matrix
        #adding regularization
        if 'nan' == pd.DataFrame(results)['z'].iloc[2].strip():
            try:
                results = ols.fit_regularized(method='l1', disp=False, maxiter=1000, alpha=0.3) #mehtod prevents singular matrix
                results = results.summary()
                converged = results.tables[0].data[5][1].strip()
                pseudo_r2 = results.tables[0].data[3][3].strip()
                results = results.tables[1].data
                results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
                results['standard_fitting'] = False        
            except:
                #crashed the regularized
                counts +=1

        results['converged'] = converged
        results['pseudo_r2'] = pseudo_r2
        results['snps'] = v
        results['order'] = len(sp)
        final_results.append(results)

    final_results = pd.concat(final_results)
    #print(counts)
    return final_results

In [9]:
# Testing all the interactions

def results_fit_to_df(results):
    coeffs = results.params.tolist()
    pvals = results.pvalues.tolist()
    pseudo_r2 = results.rsquared
    tvals = results.tvalues.tolist()
    cint_low = results.conf_int()[0].tolist()
    cint_high = results.conf_int()[1].tolist()

    results = results.summary()
    converged = results.tables[0].data[5][1].strip()
    results = results.tables[1].data
    results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
    results['P>|z|'] = pvals
    results['z'] = tvals 
    results['coef'] = coeffs
    results['converged'] = converged
    results['pseudo_r2'] = pseudo_r2
    results['[0.025'] = cint_low
    results['0.975]'] = cint_high
    return results
    
def test_interactions_high(df, data, max_order=4):
    """
    I use GLM because:
    The main difference between the two approaches is that the general linear model strictly assumes that
    the residuals will follow a conditionally normal distribution, while the GLM loosens this assumption 
    and allows for a variety of other distributions from the exponential family for the residuals.
    """
    final_results = []
    counts = 0

    for m_or in range(2,max_order+1):
        print('current order',m_or)
        
        for v in df[(df.repetitions>=2) & (df.order==m_or)].variants.tolist():
            #preparing the input
            sp=v.split('+')
            xy = data[sp+['ln_IC50']].fillna(-1)
            sp=v.replace('_','').split('+')
            xy.columns = [''.join([chr(int(y)+97) if y.isnumeric() else y for y in x.replace('_','').replace('.','')]) for x in xy.columns]
            formula = xy.columns[-1]+' ~ '
            for i in range(1,len(xy.columns)):
                formula = formula + ' + '.join(['*'.join(o) for o in list(combinations(xy.columns[:-1],i))])
                formula = formula + ' + '
            formula = formula.rstrip(' + ')
            
            #Recreating the formula
            if m_or>2:
                #gathering all interactions
                fs = formula.split(' + ')
                formula = ' + '.join([a for a in fs if '*' not in a]+[a for a in fs if a.count('*')== m_or-1])
                all_interactions = [a.replace('*',':') for a in fs if '*' in a]
                final_results = pd.concat(final_results)
                subset = final_results[final_results.coef_id.apply(lambda a: a in all_interactions)].reset_index(drop=True)
                final_results = [final_results]
                if len(subset)>0:
                    max_idx = subset['coef'].astype(float).idxmax()
                    coef_id = subset.loc[max_idx].coef_id
                    formula = formula +' + '+coef_id.replace(':','*')
                else:
                    #pass
                    continue # bc i dont think it is a valid tree form (interaction-wise)
                    #There is no sub epistasis (P>Q>O>P, tree 503, first compound)

            # Standard fitting
            try:
                ols = smf.ols(formula.replace('*',':'),data=xy)
                # "*" vs ":" #https://stackoverflow.com/questions/33050104/difference-between-the-interaction-and-term-for-formulas-in-statsmodels-ols
            except:
                print('error in OLS')
                print('coef_id',coef_id)
                print('formula OLS',type(formula),formula)
                #return pd.concat(final_results)
                continue
            ols.raise_on_perfect_prediction = False #preventing the perfect separation error
            results = ols.fit(disp=False, maxiter=1000) #mehtod prevents singular matrix
#            return results
            results = results_fit_to_df(results)
            results['standard_fitting'] = True

            #If nan means no convergence bc singular matrix
            #adding regularization
            if 'nan' == pd.DataFrame(results)['z'].astype(str).iloc[2].strip():
                try:
                    results = ols.fit_regularized(method='l1', disp=False, maxiter=1000, alpha=0.3) #mehtod prevents singular matrix
                    results = results_fit_to_df(results)
                    results['standard_fitting'] = False        
                except:
                    #crashed the regularized
                    counts +=1
                    continue


            results['snps'] = v
            results['order'] = len(sp)
            final_results.append(results)

    final_results = pd.concat(final_results)
    return final_results


In [10]:
def undo(string):
    
    string = ''.join([ x if ord(x)<90 else str(ord(x)-97) for x in string ])
    string = string[:6]+'.'+string[6:].replace('HUMAN', '_HUMAN') #not sure these 6
    return string
    
undo('PacfbbCRYABHUMAN')

'P02511.CRYAB_HUMAN'

In [11]:
#TODO: By compound

#Looping over all drugs
# drug_id
# Other options:
# - drug_name
# - CHEMBL = Chemical compound ID
#for compound_name, group in x.merge(y, left_on='Cell_Line', right_on='cell_line_name').groupby('drug_id'): # may require too much memory

#Making a temp file to run all R stuff
! mkdir tmp

#Make a folder to save all outputs
! mkdir output

column_to_group = 'drug_id'
drugs_list = y[column_to_group].unique()
i = -1
start_time = time.time()
end_time = time.time()
all_drug_results = []
for elm in drugs_list[:5]:
    i+=1
    
    if i%10==0 or i<10: print(i,elm, end_time - start_time)
    start_time = time.time()

    xy = x.merge(y[y[column_to_group]==elm], left_on='Cell_Line', right_on='cell_line_name')
    #Enhancement: Remove peptides that are all zero 
    
    # saving csv for R df
    # file name is generic but we could personalize it
    xy.drop(columns=['Cell_Line', 'cell_line_name','drug_id']).rename(columns={'ln_IC50':'label'}).to_csv("tmp/data.csv", index=False)

    #Run the R script to generate the outputs
    ! Rscript ranger_run.R
    
    #load the R outputs (the trees, one file each), and convert it to VS look-alike and get interactions
    interactions = {}
    trees = os.listdir('output/')
    #files = [x for x in files if 'tree' in x]
    for tree in trees:
        if 'tree' not in tree: continue #if it is not a tree file ignore
        tree_json = from_table_to_json(pd.read_csv('output/'+tree))        
        get_interactions(tree_json,[],interactions) #the interactions are found in "interactions"
        
    # Creating a df out of the interactions
    df = pd.DataFrame({'variants':interactions.keys(),'repetitions':interactions.values()})
    df['order'] = df.variants.apply(lambda x: x.count('+')+1)
    
    
    tested_interactions = test_interactions_high(df, xy) #here you define which order of interactions you want to compute
    tested_interactions['drug'] = elm
    all_drug_results.append(tested_interactions)
    end_time = time.time()

    

final_results = pd.concat(all_drug_results)

mkdir: tmp: File exists
mkdir: output: File exists
0 1409 1.2159347534179688e-05
current order 2


/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


current order 3
current order 4
1 1057 200.8323619365692
current order 2
current order 3
current order 4
2 1060 596.8045341968536
current order 2


/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


current order 3
current order 4
3 252 566.8235251903534
current order 2
current order 3
current order 4
4 282 179.82224893569946
current order 2


/opt/homebrew/anaconda3/envs/vs05/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


current order 3
current order 4


In [12]:
#How many tested interactions for each order wyou have
final_results[final_results.coef_id=='Intercept'].groupby('order').size().sort_values()

order
4      172
3      177
2    39470
dtype: int64

array([1409, 1057, 1060,  252,  282,  283,  346,  439,  226,  461, 1149,
       1494,  223,    1, 1001, 1004, 1005, 1006, 1007, 1008, 1009, 1010,
       1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021,
       1022, 1023, 1024, 1025, 1026, 1028, 1029, 1030, 1031, 1032, 1033,
       1036, 1037, 1038, 1039,  104, 1042, 1043, 1046, 1047, 1048, 1049,
       1050, 1052, 1053, 1054, 1058, 1059,  106, 1061, 1062, 1066, 1067,
       1069, 1072, 1091,   11,  110,  111, 1114, 1129, 1133, 1135, 1136,
       1170, 1175, 1184, 1185,  119, 1192, 1194, 1199, 1218, 1219, 1230,
       1236, 1239, 1241, 1242, 1243, 1248, 1259, 1261, 1262, 1263, 1264,
       1266, 1268,  127,  133,  134,  135,  136, 1371, 1373, 1374, 1375,
       1376, 1377, 1378, 1382, 1386, 1392, 1393, 1394,  140, 1401, 1402,
       1403, 1407, 1413, 1414, 1416, 1419, 1421, 1422, 1425, 1426, 1427,
       1428, 1429, 1430, 1432, 1433, 1434, 1437, 1441, 1444, 1445, 1446,
       1449, 1451, 1453, 1458, 1459, 1460, 1461, 14